In [5]:
%load_ext autoreload
%autoreload 2
import bert
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import spike_queries
import spacy
from typing import List
from collections import defaultdict
import tqdm
from termcolor import colored
from collections import Counter
import pandas as pd
from pybart.api import Converter
from typing import List

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:


class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'



In [7]:
def load_results(fname):
    
    with open(fname, "r", encoding = "utf-8") as f:
        sents = f.readlines()
    
    sents = [s.strip().split("\t")[-1] for s in sents]
    original, results = sents[0], sents[1:]
    return original, results


def get_spike_results_arguments_representations(model, spike_results, layers):
    
    sents = spike_results["sentence_text"].tolist()
    arg1_idx_start = spike_results["arg1_first_index"].to_numpy().astype(int)
    arg2_idx_start = spike_results["arg2_first_index"].to_numpy().astype(int)
    arg1_idx_end = spike_results["arg1_last_index"].to_numpy().astype(int)
    arg2_idx_end = spike_results["arg2_last_index"].to_numpy().astype(int)
    
    arg1_rep = []
    arg2_rep = []
    
    for s, arg1_start, arg2_start, arg1_end, arg2_end in zip(sents, arg1_idx_start, arg2_idx_start, arg1_idx_end, arg2_idx_end):
        #idx_to_mask = [arg1_start, arg2_start, arg1_end, arg2_end]
        H, _, _, orig2tok = model.encode(s, layers = layers)

        h1, h2 = H[orig2tok[arg1_start]:orig2tok[arg1_end] + 1], H[orig2tok[arg2_start]:orig2tok[arg2_end] + 1]
        
        h1 = np.mean(h1, axis = 0)
        h2 = np.mean(h2, axis = 0)
        
        arg1_rep.append(h1)
        arg2_rep.append(h2)
        
    arg1_mean = np.mean(arg1_rep, axis = 0)
    arg2_mean = np.mean(arg2_rep, axis = 0)
        
    return arg1_mean, arg2_mean
    


def main(filename, layers = [-1], num = 500):
    
    
    query, results1 = load_results(filename)
    results1 = results1[:num]
    
    spike_results = spike_queries.perform_query(query, dataset_name = "covid19", num_results = 100, query_type = "syntactic")
    spike_results = spike_results[spike_results['sentence_text'].notnull()]
    arg1_rep, arg2_rep = get_spike_results_arguments_representations(model, spike_results, layers)
        
    first, first_ind1, first_ind2 = spike_results["sentence_text"].tolist()[-1], int(spike_results["arg1_first_index"].tolist()[-1]), int(spike_results["arg2_first_index"].tolist()[-1])
    
    alignments = []
    
    for s in results1:
                    
        H, tokenized_text, tok_to_orig_map, orig2tok = model.encode(s, layers = layers)
        sims_arg1 = cosine_similarity([arg1_rep], H[1:-2])[0]
        sims_arg2 = cosine_similarity([arg2_rep], H[1:-2])[0]
        arg1_ind = np.argmax(sims_arg1) + 1
        
        sims_arg2[np.argmax(sims_arg1)] = -1
        arg2_ind = np.argmax(sims_arg2) + 1
        #print(arg1_ind, arg2_ind)
        if arg1_ind not in tok_to_orig_map or arg2_ind not in tok_to_orig_map:
            #print("ERROR")
            continue
        alignments.append((s, tok_to_orig_map[arg1_ind], tok_to_orig_map[arg2_ind]))
            
    return query, spike_results, alignments

In [8]:
model = bert.BertEncoder("cuda", "scibert")

loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/allenai/scibert_scivocab_uncased/config.json from cache at /home/shauli/.cache/torch/transformers/199e28e62d2210c23d63625bd9eecc20cf72a156b29e2a540d4933af4f50bda1.4b6b9f5d813f7395e7ea533039e02deb1723d8fd9d8ba655391a01a69ad6223d
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 31090
}

loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/allenai/scibert_scivocab_uncased/config.json from cache at /home/shauli/.cache/torch/transformers/199e28e62d2210c23d63625bd9eecc20cf72a156b29e2a540d4933af4f50bda1.4b6b9f5d8

## Parse

In [9]:
def parse_results(sentences: List[str]):
    
    #nlp = spacy.load("en_core_sci_sm")
    nlp = spacy.load("en_core_sci_lg")
    converter = Converter()
    nlp.add_pipe(converter, name="BART")

    docs = [spacy.tokens.Doc(vocab=nlp.vocab, words = sentence.split(" ")) for sentence in sentences]
    for name, proc in nlp.pipeline:
        for i,doc in enumerate(docs):
            docs[i] = proc(doc)
    return docs



def get_ancestors(tok):
    
    parents = [tok]
    
    while tok.head != tok:
    
        parents.append(tok.head)
        tok = tok.head
        
    
    return parents
    
def get_path_between_tokens(tok1, tok2, doc):
    
    path1 = get_ancestors(tok1)
    path2 = get_ancestors(tok2)
    #print(path1, path2)
    
    lowest_ancestor = None
    for tok in path1:
        if tok in path2:
            lowest_ancestor = tok
            break

    path1 =  path1[:path1.index(lowest_ancestor)+1]
    path2 =  path2[:path2.index(lowest_ancestor)+1]
    
    linear_path_before = [tok1 if tok1.i < tok2.i else tok2]
    linear_path_after = [tok1 if tok1.i > tok2.i else tok2]
    linear_path_between =  [t for t in doc if t.i < max(tok1.i, tok2.i) and t.i > min(tok1.i, tok2.i)]
    
    min_i, max_i = min(tok1.i, lowest_ancestor.i, tok2.i), max(tok1.i, lowest_ancestor.i, tok2.i)
    
    linear_path = [t.text for t in doc if t.i >= min_i and t.i <= max_i]
    linear_path[linear_path.index(tok1.text)] = colored(tok1.text, "red")
    linear_path[linear_path.index(tok2.text)] = colored(tok2.text, "blue")    
    linear_path_str = " ".join(linear_path)
    
    path = {"ancestor": lowest_ancestor, "path1": path1, "path2": path2, "tok1": tok1, "tok2": tok2, "linear_path": linear_path,
           "linear_path_str": linear_path_str}
    return path

In [10]:
#from spacy import displacy
#displacy.render(doc, style='dep', jupyter=True)

In [19]:
def regularize(path1: List[str], path2: List[str]):

    #print("path1", path1)
    #print("path2", path2)
    path_original = path1.copy() + path2[::-1][1:].copy()

    
    path1[-1] = color.BOLD + path1[-1] + color.END
    path2[0] = colored(path2[0], "blue")
    path1[0] = colored(path1[0], "red")
    
    path = path1 + path2[::-1][1:]
    
    

    return path, path_original
        

def get_paths(docs, alignments, path_type = "lemmas-deps", parse="bart"):

    paths_lemmas = []
    paths_deps = []
    paths_lemmas_deps = []
    paths_lemmas_deps_root = []
    paths_pos = []
    i = 0
    linear_paths = []
    
    for doc, (sent, idx1, idx2) in zip(docs, alignments):
        
        tok1, tok2 = doc[idx1], doc[idx2]
        
        try:
            path_dict = get_path_between_tokens(tok1, tok2, doc)
        except: continue #TODO: check the "None" error.
            
        linear_paths.append(path_dict["linear_path_str"])
    
        tok1_2ances = path_dict["path1"]
        tok2_2ances = path_dict["path2"]
        
        
        lemmas1 = [tok.lemma_ for tok in tok1_2ances]
        lemmas2 = [tok.lemma_ for tok in tok2_2ances]
        
        if parse == "pybart":
            deps1 = [tok._.parent_list[0]["rel"] for tok in tok1_2ances]
            deps2 = [tok._.parent_list[0]["rel"] for tok in tok2_2ances]            
        else:
            deps1 = [tok.dep_ for tok in tok1_2ances]
            deps2 = [tok.dep_ for tok in tok2_2ances]
        #pos1 = [tok.tag_ for tok in tok1_2ances]
        #pos2 = [tok.tag_ for tok in tok2_2ances]
        
        lemmas, lemmas_original = regularize(lemmas1, lemmas2)
        deps, deps_original = regularize(deps1, deps2)
        #pos = regularize(pos1, pos2)
        
        zipped = "-".join([lemma+"."+dep for lemma, dep in zip(lemmas, deps)])
        assert len(lemmas) == len(deps) == len(lemmas_original)
        
        zipped_only_root = "-".join([lemma+"."+dep if lemma_orig==path_dict["ancestor"].lemma_ else dep for lemma, dep,lemma_orig in zip(lemmas, deps, lemmas_original)])
        paths_lemmas_deps.append(zipped)
        paths_lemmas.append("-".join(lemmas))
        paths_deps.append("-".join(deps))
        paths_lemmas_deps_root.append(zipped_only_root)
        #paths_pos.append("-".join(paths_pos))
        
        i += 1
    
    paths = None
    
    if path_type == "lemmas-deps":
        
        paths =  paths_lemmas_deps
    
    elif path_type == "lemmas-deps-root":
        
        paths = paths_lemmas_deps_root

    elif path_type == "lemmas":
        
        paths = paths_lemmas
    
    elif path_type == "deps":
        
        paths = paths_deps
    
    elif path_type == "pos":
        
        paths = paths_pos
    
    else:
        
        raise Exception("Unknown type")
    
    return paths, linear_paths

In [12]:

def sort_by_freq(paths, linear_paths):
    
    together = zip(paths, linear_paths)
    counts = []
    
    for p, p_lin in zip(paths, linear_paths):
        
        counts.append(paths.count(p))

    paths_linpaths_counts = zip(paths, linear_paths, counts)    
    paths_linpaths_counts = sorted(paths_linpaths_counts, key = lambda pair: -pair[-1])
    
    paths, linear_paths, counts = list(zip(*paths_linpaths_counts))
    return paths, linear_paths, counts
    

# Collect Paths

In [13]:
def collect(num_results, path_type, parse_type="dep"):
    
    fnames = ["results{}.txt".format(i) for i in range(1,10)]
    k = 0
    
    for fname in fnames:
        query, spike_results, alignments = main(fname, layers = [-1], num = num_results)    
        sents = [s for s,ind1,ind2 in alignments]
        docs = parse_results(sents)
    
        paths, linear_paths = get_paths(docs, alignments, path_type, parse_type)
        paths, linear_paths, counts = sort_by_freq(paths, linear_paths)
    
        #df = pd.DataFrame({"paths": paths, "linear_paths": linear_paths})
    
        print(color.BOLD + "QUERY {}:\n\t".format(k) + query + color.END )
        print(color.BOLD + "\nMOST FREQUENT PATTERNS:\n" + color.END)
        for i in range(150):
        
            print("PATH: "+ paths[i] + "\tLINEAR: " + linear_paths[i] +"\tCOUNT: " + str(counts[i]))
        
        print("\n\n")
        print("=================================================================")
        print("=================================================================")    
        
        k += 1

## Pattern: dep labels + lemmas

In [62]:
collect(500, "lemmas-deps")

QUERY 0:
	<>arg1:virus $infection $causes a <>arg2:condition .

MOST FREQUENT PATTERNS:

PATH: infection.nsubj-cause.ROOT-disease.dobj	LINEAR: infections cause a debilitating disease	COUNT: 7
PATH: infection.nsubj-cause.ROOT-disease.dobj	LINEAR: infection causes respiratory tract disease	COUNT: 7
PATH: infection.nsubj-cause.ROOT-disease.dobj	LINEAR: infection of JEV can cause nervous system disease	COUNT: 7
PATH: infection.nsubj-cause.ROOT-disease.dobj	LINEAR: Infection with LIV in humans can cause a neurological disease	COUNT: 7
PATH: infection.nsubj-cause.ROOT-disease.dobj	LINEAR: Infection with SARS-CoV causes lower respiratory tract disease	COUNT: 7
PATH: infection.nsubj-cause.ROOT-disease.dobj	LINEAR: Infection with MAdV-1 causes multisystemic disease	COUNT: 7
PATH: infection.nsubj-cause.ROOT-disease.dobj	LINEAR: infection can cause severe lung disease	COUNT: 7
PATH: infection.nsubj-result.ROOT-disease.nmod	LINEAR: infection can result in severe respiratory disease	COUNT: 6
PATH: 

QUERY 1:
	infected arg1:patients $usually $develop <>arg2:[entity]respiratory illness.

MOST FREQUENT PATTERNS:

PATH: infection.nsubjpass-characterize.ROOT-be.auxpass	LINEAR: Infection with PEDV is characterized	COUNT: 9
PATH: infection.nsubjpass-characterize.ROOT-be.auxpass	LINEAR: Infection is characterized	COUNT: 9
PATH: infection.nsubjpass-characterize.ROOT-be.auxpass	LINEAR: infection is clinically characterized	COUNT: 9
PATH: infection.nsubjpass-characterize.ROOT-be.auxpass	LINEAR: infection is characterized	COUNT: 9
PATH: infection.nsubjpass-characterize.ROOT-be.auxpass	LINEAR: infection is characterized	COUNT: 9
PATH: infection.nsubjpass-characterize.ROOT-be.auxpass	LINEAR: infection is characterized	COUNT: 9
PATH: infection.nsubjpass-characterize.ROOT-be.auxpass	LINEAR: infection is characterized	COUNT: 9
PATH: infection.nsubjpass-characterize.ROOT-be.auxpass	LINEAR: infection is characterized	COUNT: 9
PATH: infection.nsubjpass-characterize.ROOT-be.auxpass	LINEAR: infections 

QUERY 2:
	a arg1:subset of patients $progress to arg2:hemorrhagic fever

MOST FREQUENT PATTERNS:

PATH: case.nsubj-severe.amod	LINEAR: Severe cases	COUNT: 9
PATH: case.nsubj-severe.amod	LINEAR: Severe cases	COUNT: 9
PATH: case.nsubj-severe.amod	LINEAR: Severe pediatric cases	COUNT: 9
PATH: case.nsubj-severe.amod	LINEAR: Severe cases	COUNT: 9
PATH: case.nsubj-severe.amod	LINEAR: severe cases	COUNT: 9
PATH: case.nsubj-severe.amod	LINEAR: severe cases	COUNT: 9
PATH: case.nsubj-severe.amod	LINEAR: Severe cases	COUNT: 9
PATH: case.nsubj-severe.amod	LINEAR: severe cases	COUNT: 9
PATH: case.nsubj-severe.amod	LINEAR: Severe cases	COUNT: 9
PATH: patient.nsubj-severe.amod	LINEAR: Severe patients	COUNT: 6
PATH: patient.nsubj-severe.amod	LINEAR: Severe patients	COUNT: 6
PATH: patient.nsubj-severe.amod	LINEAR: Severe patients	COUNT: 6
PATH: patient.nsubj-severe.amod	LINEAR: Severe patients	COUNT: 6
PATH: patient.nsubj-severe.amod	LINEAR: Severe patients	COUNT: 6
PATH: patient.nsubj-severe.amod	LINE

QUERY 3:
	people with arg1:[e]asthma are $susceptible to arg2:[e]asthma

MOST FREQUENT PATTERNS:

PATH: copd.nmod-subject.conj-exacerbation.dobj-trigger.xcomp-know.ROOT-infection.nsubjpass	LINEAR: Infection with RV is known to trigger acute exacerbations and subjects with COPD	COUNT: 14
PATH: copd.nmod-subject.conj-exacerbation.dobj-trigger.xcomp-know.ROOT-infection.nsubjpass	LINEAR: Infection with RV is known to trigger acute exacerbations and subjects with COPD	COUNT: 14
PATH: copd.nmod-subject.conj-exacerbation.dobj-trigger.xcomp-know.ROOT-infection.nsubjpass	LINEAR: Infection with RV is known to trigger acute exacerbations and subjects with COPD	COUNT: 14
PATH: copd.nmod-subject.conj-exacerbation.dobj-trigger.xcomp-know.ROOT-infection.nsubjpass	LINEAR: Infection with RV is known to trigger acute exacerbations and subjects with COPD	COUNT: 14
PATH: copd.nmod-subject.conj-exacerbation.dobj-trigger.xcomp-know.ROOT-infection.nsubjpass	LINEAR: Infection with RV is known to trigger acute

QUERY 4:
	arg1:[e]paracetamol is $not useful for treating arg2:[e]asthma.

MOST FREQUENT PATTERNS:

PATH: corticosteroid.nsubj-increase.ROOT-influenza.nmod-infection.conj	LINEAR: corticosteroids may increase the risk of mortality and morbidity ( e.g. secondary infections ) in severe influenza and other viral infections	COUNT: 2
PATH: corticosteroid.nsubj-increase.ROOT-influenza.nmod-infection.conj	LINEAR: corticosteroids may increase the risk of mortality and morbidity ( e.g. secondary infections ) in severe influenza and other viral infections	COUNT: 2
PATH: ribavirin.nsubjpass-consider.ccomp-management.nmod-case.nmod-severe.amod	LINEAR: ribavirin and IgIV can be considered for the management of severe	COUNT: 2
PATH: ribavirin.nsubj-effective.ROOT-virus.nmod-sars.compound	LINEAR: ribavirin is only marginally effective against the SARS	COUNT: 2
PATH: antiviral.amod-option.nsubjpass-limited.ROOT-be.advcl-prove.conj-effective.xcomp-occur.advcl-disease.nsubj	LINEAR: antiviral treatment op

QUERY 5:
	arg1:[e]stroke is a $complication of :[e]COVID-19 arg2:infection

MOST FREQUENT PATTERNS:

PATH: disease.ROOT-be.cop	LINEAR: is a novel respiratory disease	COUNT: 7
PATH: disease.ROOT-be.cop	LINEAR: is a novel respiratory disease	COUNT: 7
PATH: disease.ROOT-be.cop	LINEAR: is a highly contagious disease	COUNT: 7
PATH: disease.ROOT-be.cop	LINEAR: is an infectious disease	COUNT: 7
PATH: disease.ROOT-be.cop	LINEAR: is an acute infectious disease	COUNT: 7
PATH: disease.ROOT-be.cop	LINEAR: is an acute resolved disease	COUNT: 7
PATH: disease.ROOT-be.cop	LINEAR: is a common cardiovascular disease	COUNT: 7
PATH: infection.nmod-patient.nsubj	LINEAR: Patients with severe COVID-19 infection	COUNT: 5
PATH: infection.nsubj-complication.ROOT-patient.nmod	LINEAR: Infection is a common complication in patients	COUNT: 5
PATH: infection.nsubj-complication.ROOT-patient.nmod	LINEAR: infection is a known complication among critically ill patients	COUNT: 5
PATH: infection.nmod-patient.nsubj	LINEAR:

QUERY 6:
	the recommended arg1:[w]quarantine period is arg2:14 :[w]days

MOST FREQUENT PATTERNS:

PATH: quarantine.xcomp-for.prep-day.pobj-14.nummod	LINEAR: quarantined for 14	COUNT: 4
PATH: quarantine.xcomp-for.prep-day.pobj-14.nummod	LINEAR: quarantined for 14	COUNT: 4
PATH: quarantine.xcomp-for.prep-day.pobj-14.nummod	LINEAR: quarantined for 14	COUNT: 4
PATH: vaccination.compound-protocol.nsubj-be.ROOT-administration.attr-single.amod	LINEAR: vaccination protocol in this situation would be a single	COUNT: 4
PATH: vaccination.compound-protocol.nsubj-be.ROOT-administration.attr-single.amod	LINEAR: vaccination protocol in this situation would be a single	COUNT: 4
PATH: vaccination.compound-protocol.nsubj-be.ROOT-administration.attr-single.amod	LINEAR: vaccination protocol in this situation would be a single	COUNT: 4
PATH: vaccination.compound-protocol.nsubj-be.ROOT-administration.attr-single.amod	LINEAR: vaccination protocol in this situation would be a single	COUNT: 4
PATH: quarantine.

QUERY 7:
	arg1:[e]COVID-19 $infects arg2:cells

MOST FREQUENT PATTERNS:

PATH: sars-cov-2.nsubj-infect.ROOT-cell.dobj	LINEAR: SARS-CoV-2 infects lung alveolar epithelial cells	COUNT: 16
PATH: sars-cov-2.nsubj-infect.ROOT-cell.dobj	LINEAR: SARS-CoV-2 infects cells	COUNT: 16
PATH: sars-cov-2.nsubj-infect.ROOT-cell.dobj	LINEAR: SARS-CoV-2 infects ACE2positive cells	COUNT: 16
PATH: sars-cov-2.nsubj-infect.ROOT-cell.dobj	LINEAR: SARS-CoV-2 infects human epithelial cells	COUNT: 16
PATH: sars-cov-2.nsubj-infect.ROOT-cell.dobj	LINEAR: SARS-CoV-2 can infect cells	COUNT: 16
PATH: sars-cov-2.nsubj-enter.ROOT-cell.dobj	LINEAR: SARS-CoV-2 enters human cells	COUNT: 16
PATH: sars-cov-2.nsubj-enter.ROOT-cell.dobj	LINEAR: SARS-CoV-2 enters respiratory epithelial cells	COUNT: 16
PATH: sars-cov-2.nsubj-enter.ROOT-cell.dobj	LINEAR: SARS-CoV-2 enters human cells	COUNT: 16
PATH: sars-cov-2.nsubj-infect.ROOT-cell.dobj	LINEAR: SARS-CoV-2 infects endothelial cells	COUNT: 16
PATH: sars-cov-2.nsubj-enter.ROOT-ce

QUERY 8:
	arg1:[e]COVID-19 activates the arg2:ATP $receptor

MOST FREQUENT PATTERNS:

PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: Enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzy

## Pattern: dep labels + lemmas (PyBART)

In [63]:
collect(500, "lemmas-deps", parse_type="pybart")

QUERY 0:
	<>arg1:virus $infection $causes a <>arg2:condition .

MOST FREQUENT PATTERNS:

PATH: infection.nsubj-cause.root-disease.dobj	LINEAR: infections cause a debilitating disease	COUNT: 7
PATH: infection.nsubj-cause.root-disease.dobj	LINEAR: infection causes respiratory tract disease	COUNT: 7
PATH: infection.nsubj-cause.root-disease.dobj	LINEAR: infection of JEV can cause nervous system disease	COUNT: 7
PATH: infection.nsubj-cause.root-disease.dobj	LINEAR: Infection with LIV in humans can cause a neurological disease	COUNT: 7
PATH: infection.nsubj-cause.root-disease.dobj	LINEAR: Infection with SARS-CoV causes lower respiratory tract disease	COUNT: 7
PATH: infection.nsubj-cause.root-disease.dobj	LINEAR: Infection with MAdV-1 causes multisystemic disease	COUNT: 7
PATH: infection.nsubj-cause.root-disease.dobj	LINEAR: infection can cause severe lung disease	COUNT: 7
PATH: infection.nsubj-result.root-disease.nmod:in	LINEAR: infection can result in severe respiratory disease	COUNT: 6
PAT

QUERY 1:
	infected arg1:patients $usually $develop <>arg2:[entity]respiratory illness.

MOST FREQUENT PATTERNS:

PATH: infection.nsubjpass-characterize.root-be.auxpass	LINEAR: Infection with PEDV is characterized	COUNT: 9
PATH: infection.nsubjpass-characterize.root-be.auxpass	LINEAR: Infection is characterized	COUNT: 9
PATH: infection.nsubjpass-characterize.root-be.auxpass	LINEAR: infection is clinically characterized	COUNT: 9
PATH: infection.nsubjpass-characterize.root-be.auxpass	LINEAR: infection is characterized	COUNT: 9
PATH: infection.nsubjpass-characterize.root-be.auxpass	LINEAR: infection is characterized	COUNT: 9
PATH: infection.nsubjpass-characterize.root-be.auxpass	LINEAR: infection is characterized	COUNT: 9
PATH: infection.nsubjpass-characterize.root-be.auxpass	LINEAR: infection is characterized	COUNT: 9
PATH: infection.nsubjpass-characterize.root-be.auxpass	LINEAR: infection is characterized	COUNT: 9
PATH: infection.nsubjpass-characterize.root-be.auxpass	LINEAR: infections 

QUERY 2:
	a arg1:subset of patients $progress to arg2:hemorrhagic fever

MOST FREQUENT PATTERNS:

PATH: case.nsubj-severe.amod	LINEAR: Severe cases	COUNT: 9
PATH: case.nsubj-severe.amod	LINEAR: Severe cases	COUNT: 9
PATH: case.nsubj-severe.amod	LINEAR: Severe pediatric cases	COUNT: 9
PATH: case.nsubj-severe.amod	LINEAR: Severe cases	COUNT: 9
PATH: case.nsubj-severe.amod	LINEAR: severe cases	COUNT: 9
PATH: case.nsubj-severe.amod	LINEAR: severe cases	COUNT: 9
PATH: case.nsubj-severe.amod	LINEAR: Severe cases	COUNT: 9
PATH: case.nsubj-severe.amod	LINEAR: severe cases	COUNT: 9
PATH: case.nsubj-severe.amod	LINEAR: Severe cases	COUNT: 9
PATH: patient.nsubj-severe.amod	LINEAR: Severe patients	COUNT: 6
PATH: patient.nsubj-severe.amod	LINEAR: Severe patients	COUNT: 6
PATH: patient.nsubj-severe.amod	LINEAR: Severe patients	COUNT: 6
PATH: patient.nsubj-severe.amod	LINEAR: Severe patients	COUNT: 6
PATH: patient.nsubj-severe.amod	LINEAR: Severe patients	COUNT: 6
PATH: patient.nsubj-severe.amod	LINE

QUERY 3:
	people with arg1:[e]asthma are $susceptible to arg2:[e]asthma

MOST FREQUENT PATTERNS:

PATH: copd.nmod:with-subject.dobj-exacerbation.dobj-trigger.xcomp-know.root-infection.nsubjpass	LINEAR: Infection with RV is known to trigger acute exacerbations and subjects with COPD	COUNT: 14
PATH: copd.nmod:with-subject.dobj-exacerbation.dobj-trigger.xcomp-know.root-infection.nsubjpass	LINEAR: Infection with RV is known to trigger acute exacerbations and subjects with COPD	COUNT: 14
PATH: copd.nmod:with-subject.dobj-exacerbation.dobj-trigger.xcomp-know.root-infection.nsubjpass	LINEAR: Infection with RV is known to trigger acute exacerbations and subjects with COPD	COUNT: 14
PATH: copd.nmod:with-subject.dobj-exacerbation.dobj-trigger.xcomp-know.root-infection.nsubjpass	LINEAR: Infection with RV is known to trigger acute exacerbations and subjects with COPD	COUNT: 14
PATH: copd.nmod:with-subject.dobj-exacerbation.dobj-trigger.xcomp-know.root-infection.nsubjpass	LINEAR: Infection with RV 

QUERY 4:
	arg1:[e]paracetamol is $not useful for treating arg2:[e]asthma.

MOST FREQUENT PATTERNS:

PATH: corticosteroid.nsubj-increase.root-influenza.nmod:in-infection.nmod:in	LINEAR: corticosteroids may increase the risk of mortality and morbidity ( e.g. secondary infections ) in severe influenza and other viral infections	COUNT: 2
PATH: corticosteroid.nsubj-increase.root-influenza.nmod:in-infection.nmod:in	LINEAR: corticosteroids may increase the risk of mortality and morbidity ( e.g. secondary infections ) in severe influenza and other viral infections	COUNT: 2
PATH: ribavirin.nsubjpass-consider.ccomp-management.nmod:for-case.nmod:of-severe.amod	LINEAR: ribavirin and IgIV can be considered for the management of severe	COUNT: 2
PATH: ribavirin.nsubj-effective.amod-virus.nmod:against-sars.compound	LINEAR: ribavirin is only marginally effective against the SARS	COUNT: 2
PATH: antiviral.amod-option.nsubjpass-limited.root-be.advcl:because-prove.advcl:because-effective.xcomp-occur.advcl:

QUERY 5:
	arg1:[e]stroke is a $complication of :[e]COVID-19 arg2:infection

MOST FREQUENT PATTERNS:

PATH: disease.xcomp-be.ev	LINEAR: is a novel respiratory disease	COUNT: 7
PATH: disease.xcomp-be.ev	LINEAR: is a novel respiratory disease	COUNT: 7
PATH: disease.xcomp-be.ev	LINEAR: is a highly contagious disease	COUNT: 7
PATH: disease.xcomp-be.ev	LINEAR: is an infectious disease	COUNT: 7
PATH: disease.xcomp-be.ev	LINEAR: is an acute infectious disease	COUNT: 7
PATH: disease.xcomp-be.ev	LINEAR: is an acute resolved disease	COUNT: 7
PATH: disease.xcomp-be.ev	LINEAR: is a common cardiovascular disease	COUNT: 7
PATH: infection.nmod:with-patient.nsubj	LINEAR: Patients with severe COVID-19 infection	COUNT: 5
PATH: infection.nmod:with-patient.nsubj	LINEAR: Patients with invasive GAS infection	COUNT: 5
PATH: infection.nmod:with-patient.nsubj	LINEAR: Patients with COVID-19 infection	COUNT: 5
PATH: infection.nmod:with-patient.nsubj	LINEAR: Patients with COVID-19 infection	COUNT: 5
PATH: infectio

QUERY 6:
	the recommended arg1:[w]quarantine period is arg2:14 :[w]days

MOST FREQUENT PATTERNS:

PATH: quarantine.xcomp-for.prep-day.pobj-14.nummod	LINEAR: quarantined for 14	COUNT: 4
PATH: quarantine.xcomp-for.prep-day.pobj-14.nummod	LINEAR: quarantined for 14	COUNT: 4
PATH: quarantine.xcomp-for.prep-day.pobj-14.nummod	LINEAR: quarantined for 14	COUNT: 4
PATH: vaccination.compound-protocol.nsubj-be.root-administration.attr-single.amod	LINEAR: vaccination protocol in this situation would be a single	COUNT: 4
PATH: vaccination.compound-protocol.nsubj-be.root-administration.attr-single.amod	LINEAR: vaccination protocol in this situation would be a single	COUNT: 4
PATH: vaccination.compound-protocol.nsubj-be.root-administration.attr-single.amod	LINEAR: vaccination protocol in this situation would be a single	COUNT: 4
PATH: vaccination.compound-protocol.nsubj-be.root-administration.attr-single.amod	LINEAR: vaccination protocol in this situation would be a single	COUNT: 4
PATH: quarantine.

QUERY 7:
	arg1:[e]COVID-19 $infects arg2:cells

MOST FREQUENT PATTERNS:

PATH: sars-cov-2.nsubj-infect.root-cell.dobj	LINEAR: SARS-CoV-2 infects lung alveolar epithelial cells	COUNT: 16
PATH: sars-cov-2.nsubj-infect.root-cell.dobj	LINEAR: SARS-CoV-2 infects cells	COUNT: 16
PATH: sars-cov-2.nsubj-infect.root-cell.dobj	LINEAR: SARS-CoV-2 infects ACE2positive cells	COUNT: 16
PATH: sars-cov-2.nsubj-infect.root-cell.dobj	LINEAR: SARS-CoV-2 infects human epithelial cells	COUNT: 16
PATH: sars-cov-2.nsubj-infect.root-cell.dobj	LINEAR: SARS-CoV-2 can infect cells	COUNT: 16
PATH: sars-cov-2.nsubj-enter.root-cell.dobj	LINEAR: SARS-CoV-2 enters human cells	COUNT: 16
PATH: sars-cov-2.nsubj-enter.root-cell.dobj	LINEAR: SARS-CoV-2 enters respiratory epithelial cells	COUNT: 16
PATH: sars-cov-2.nsubj-enter.root-cell.dobj	LINEAR: SARS-CoV-2 enters human cells	COUNT: 16
PATH: sars-cov-2.nsubj-infect.root-cell.dobj	LINEAR: SARS-CoV-2 infects endothelial cells	COUNT: 16
PATH: sars-cov-2.nsubj-enter.root-ce

QUERY 8:
	arg1:[e]COVID-19 activates the arg2:ATP $receptor

MOST FREQUENT PATTERNS:

PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: Enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzyme 2 ( ACE2	COUNT: 16
PATH: 2.nummod-enzyme.dobj-ace2.appos	LINEAR: enzy

## Pattern: lemmas along dep path

In [64]:
collect(500, "lemmas")

QUERY 0:
	<>arg1:virus $infection $causes a <>arg2:condition .

MOST FREQUENT PATTERNS:

PATH: infection-cause-disease	LINEAR: infections cause a debilitating disease	COUNT: 8
PATH: infection-cause-disease	LINEAR: infection causes respiratory tract disease	COUNT: 8
PATH: infection-cause-disease	LINEAR: infection in humans can cause a wide range of clinical manifestations , from mild febrile disease	COUNT: 8
PATH: infection-cause-disease	LINEAR: infection of JEV can cause nervous system disease	COUNT: 8
PATH: infection-cause-disease	LINEAR: Infection with LIV in humans can cause a neurological disease	COUNT: 8
PATH: infection-cause-disease	LINEAR: Infection with SARS-CoV causes lower respiratory tract disease	COUNT: 8
PATH: infection-cause-disease	LINEAR: Infection with MAdV-1 causes multisystemic disease	COUNT: 8
PATH: infection-cause-disease	LINEAR: infection can cause severe lung disease	COUNT: 8
PATH: infection-result-disease	LINEAR: infection can result in severe respiratory diseas

QUERY 1:
	infected arg1:patients $usually $develop <>arg2:[entity]respiratory illness.

MOST FREQUENT PATTERNS:

PATH: infection-characterize-be	LINEAR: Infection with PEDV is characterized	COUNT: 9
PATH: infection-characterize-be	LINEAR: Infection is characterized	COUNT: 9
PATH: infection-characterize-be	LINEAR: infection is clinically characterized	COUNT: 9
PATH: infection-characterize-be	LINEAR: infection is characterized	COUNT: 9
PATH: infection-characterize-be	LINEAR: infection is characterized	COUNT: 9
PATH: infection-characterize-be	LINEAR: infection is characterized	COUNT: 9
PATH: infection-characterize-be	LINEAR: infection is characterized	COUNT: 9
PATH: infection-characterize-be	LINEAR: infection is characterized	COUNT: 9
PATH: infection-characterize-be	LINEAR: infections are characterized	COUNT: 9
PATH: infection-sign-clinical	LINEAR: Clinical signs of CPV-2 infection	COUNT: 5
PATH: infection-sign-clinical	LINEAR: Clinical signs in human infections	COUNT: 5
PATH: infection-s

QUERY 2:
	a arg1:subset of patients $progress to arg2:hemorrhagic fever

MOST FREQUENT PATTERNS:

PATH: case-severe	LINEAR: Severe cases	COUNT: 12
PATH: case-severe	LINEAR: Severe cases	COUNT: 12
PATH: case-severe	LINEAR: Severe pediatric cases	COUNT: 12
PATH: case-severe	LINEAR: severe cases	COUNT: 12
PATH: case-severe	LINEAR: Severe cases	COUNT: 12
PATH: case-severe	LINEAR: severe cases	COUNT: 12
PATH: case-severe	LINEAR: severe cases	COUNT: 12
PATH: case-severe	LINEAR: severe cases	COUNT: 12
PATH: case-severe	LINEAR: Severe cases	COUNT: 12
PATH: case-severe	LINEAR: severe cases	COUNT: 12
PATH: case-severe	LINEAR: Severe cases	COUNT: 12
PATH: case-severe	LINEAR: severe cases	COUNT: 12
PATH: patient-severe	LINEAR: severe patients	COUNT: 7
PATH: patient-severe	LINEAR: Severe patients	COUNT: 7
PATH: patient-severe	LINEAR: Severe patients	COUNT: 7
PATH: patient-severe	LINEAR: Severe patients	COUNT: 7
PATH: patient-severe	LINEAR: Severe patients	COUNT: 7
PATH: patient-severe	LINEAR: Sever

QUERY 3:
	people with arg1:[e]asthma are $susceptible to arg2:[e]asthma

MOST FREQUENT PATTERNS:

PATH: cancer-patient-susceptible-infection	LINEAR: cancer are highly susceptible to infection	COUNT: 20
PATH: cancer-patient-susceptible-infection	LINEAR: cancer are highly susceptible to infection	COUNT: 20
PATH: cancer-patient-susceptible-infection	LINEAR: Cancer patients , particularly those under anticancer treatment , are potentially more susceptible for a severe infection	COUNT: 20
PATH: cancer-patient-susceptible-infection	LINEAR: cancer are more susceptible to the infection	COUNT: 20
PATH: cancer-patient-susceptible-infection	LINEAR: Cancer patients and immunocompromised hosts are particularly susceptible to lung infection	COUNT: 20
PATH: cancer-patient-susceptible-infection	LINEAR: cancer are more susceptible to infection	COUNT: 20
PATH: cancer-patient-susceptible-infection	LINEAR: cancer are generally more susceptible to infections	COUNT: 20
PATH: cancer-patient-susceptible-infec

QUERY 4:
	arg1:[e]paracetamol is $not useful for treating arg2:[e]asthma.

MOST FREQUENT PATTERNS:

PATH: corticosteroid-increase-influenza-infection	LINEAR: corticosteroids may increase the risk of mortality and morbidity ( e.g. secondary infections ) in severe influenza and other viral infections	COUNT: 2
PATH: corticosteroid-increase-influenza-infection	LINEAR: corticosteroids may increase the risk of mortality and morbidity ( e.g. secondary infections ) in severe influenza and other viral infections	COUNT: 2
PATH: ribavirin-consider-management-case-severe	LINEAR: ribavirin and IgIV can be considered for the management of severe	COUNT: 2
PATH: ribavirin-effective-virus-sars	LINEAR: ribavirin is only marginally effective against the SARS	COUNT: 2
PATH: antiviral-option-limited-be-prove-effective-occur-disease	LINEAR: antiviral treatment options are limited because no parenteral drug is available and no drug has been proved to be effective once life-threatening disease	COUNT: 2
PATH: 

QUERY 5:
	arg1:[e]stroke is a $complication of :[e]COVID-19 arg2:infection

MOST FREQUENT PATTERNS:

PATH: disease-be	LINEAR: is a novel respiratory disease	COUNT: 7
PATH: disease-be	LINEAR: is a novel respiratory disease	COUNT: 7
PATH: disease-be	LINEAR: is a highly contagious disease	COUNT: 7
PATH: disease-be	LINEAR: is an infectious disease	COUNT: 7
PATH: disease-be	LINEAR: is an acute infectious disease	COUNT: 7
PATH: disease-be	LINEAR: is an acute resolved disease	COUNT: 7
PATH: disease-be	LINEAR: is a common cardiovascular disease	COUNT: 7
PATH: infection-patient	LINEAR: Patients with severe COVID-19 infection	COUNT: 5
PATH: infection-complication-patient	LINEAR: Infection is a common complication in patients	COUNT: 5
PATH: infection-complication-patient	LINEAR: infection is a known complication among critically ill patients	COUNT: 5
PATH: infection-patient	LINEAR: Patients with invasive GAS infection	COUNT: 5
PATH: infection-complication-patient	LINEAR: infection ( varicella ) i

QUERY 6:
	the recommended arg1:[w]quarantine period is arg2:14 :[w]days

MOST FREQUENT PATTERNS:

PATH: quarantine-for-day-14	LINEAR: quarantined for 14	COUNT: 7
PATH: quarantine-for-day-14	LINEAR: quarantined for 14	COUNT: 7
PATH: quarantine-for-day-14	LINEAR: quarantined for 14	COUNT: 7
PATH: quarantine-for-day-14	LINEAR: quarantined for 14	COUNT: 7
PATH: quarantine-for-day-14	LINEAR: quarantined for 14	COUNT: 7
PATH: quarantine-for-day-14	LINEAR: quarantined for 14	COUNT: 7
PATH: quarantine-for-day-14	LINEAR: quarantine for 14	COUNT: 7
PATH: quarantine-period	LINEAR: quarantine period	COUNT: 6
PATH: quarantine-a	LINEAR: a 14-day quarantine	COUNT: 6
PATH: quarantine-period	LINEAR: quarantine period	COUNT: 6
PATH: quarantine-a	LINEAR: A 14-day quarantine	COUNT: 6
PATH: quarantine-a	LINEAR: A 14‐day quarantine	COUNT: 6
PATH: quarantine-period	LINEAR: quarantine period	COUNT: 6
PATH: quarantine-a	LINEAR: a 2-week quarantine	COUNT: 6
PATH: quarantine-a	LINEAR: a 14-day quarantine	COUNT: 

QUERY 7:
	arg1:[e]COVID-19 $infects arg2:cells

MOST FREQUENT PATTERNS:

PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 infects lung alveolar epithelial cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 infects cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 infects ACE2positive cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 infects human epithelial cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 can infect cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 infects endothelial cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 infects host cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 infects host cell	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 infects cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-COV-2 infects human cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 mainly infects type II alveolar epithelial cells	COUNT: 21
PATH: sars-cov-

QUERY 8:
	arg1:[e]COVID-19 activates the arg2:ATP $receptor

MOST FREQUENT PATTERNS:

PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: Enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT:

## Pattern: lemmas along dep path (PyBART)

In [65]:
collect(500, "lemmas", parse_type="pybart")

QUERY 0:
	<>arg1:virus $infection $causes a <>arg2:condition .

MOST FREQUENT PATTERNS:

PATH: infection-cause-disease	LINEAR: infections cause a debilitating disease	COUNT: 8
PATH: infection-cause-disease	LINEAR: infection causes respiratory tract disease	COUNT: 8
PATH: infection-cause-disease	LINEAR: infection in humans can cause a wide range of clinical manifestations , from mild febrile disease	COUNT: 8
PATH: infection-cause-disease	LINEAR: infection of JEV can cause nervous system disease	COUNT: 8
PATH: infection-cause-disease	LINEAR: Infection with LIV in humans can cause a neurological disease	COUNT: 8
PATH: infection-cause-disease	LINEAR: Infection with SARS-CoV causes lower respiratory tract disease	COUNT: 8
PATH: infection-cause-disease	LINEAR: Infection with MAdV-1 causes multisystemic disease	COUNT: 8
PATH: infection-cause-disease	LINEAR: infection can cause severe lung disease	COUNT: 8
PATH: infection-result-disease	LINEAR: infection can result in severe respiratory diseas

QUERY 1:
	infected arg1:patients $usually $develop <>arg2:[entity]respiratory illness.

MOST FREQUENT PATTERNS:

PATH: infection-characterize-be	LINEAR: Infection with PEDV is characterized	COUNT: 9
PATH: infection-characterize-be	LINEAR: Infection is characterized	COUNT: 9
PATH: infection-characterize-be	LINEAR: infection is clinically characterized	COUNT: 9
PATH: infection-characterize-be	LINEAR: infection is characterized	COUNT: 9
PATH: infection-characterize-be	LINEAR: infection is characterized	COUNT: 9
PATH: infection-characterize-be	LINEAR: infection is characterized	COUNT: 9
PATH: infection-characterize-be	LINEAR: infection is characterized	COUNT: 9
PATH: infection-characterize-be	LINEAR: infection is characterized	COUNT: 9
PATH: infection-characterize-be	LINEAR: infections are characterized	COUNT: 9
PATH: infection-sign-clinical	LINEAR: Clinical signs of CPV-2 infection	COUNT: 5
PATH: infection-sign-clinical	LINEAR: Clinical signs in human infections	COUNT: 5
PATH: infection-s

QUERY 2:
	a arg1:subset of patients $progress to arg2:hemorrhagic fever

MOST FREQUENT PATTERNS:

PATH: case-severe	LINEAR: Severe cases	COUNT: 12
PATH: case-severe	LINEAR: Severe cases	COUNT: 12
PATH: case-severe	LINEAR: Severe pediatric cases	COUNT: 12
PATH: case-severe	LINEAR: severe cases	COUNT: 12
PATH: case-severe	LINEAR: Severe cases	COUNT: 12
PATH: case-severe	LINEAR: severe cases	COUNT: 12
PATH: case-severe	LINEAR: severe cases	COUNT: 12
PATH: case-severe	LINEAR: severe cases	COUNT: 12
PATH: case-severe	LINEAR: Severe cases	COUNT: 12
PATH: case-severe	LINEAR: severe cases	COUNT: 12
PATH: case-severe	LINEAR: Severe cases	COUNT: 12
PATH: case-severe	LINEAR: severe cases	COUNT: 12
PATH: patient-severe	LINEAR: severe patients	COUNT: 7
PATH: patient-severe	LINEAR: Severe patients	COUNT: 7
PATH: patient-severe	LINEAR: Severe patients	COUNT: 7
PATH: patient-severe	LINEAR: Severe patients	COUNT: 7
PATH: patient-severe	LINEAR: Severe patients	COUNT: 7
PATH: patient-severe	LINEAR: Sever

QUERY 3:
	people with arg1:[e]asthma are $susceptible to arg2:[e]asthma

MOST FREQUENT PATTERNS:

PATH: cancer-patient-susceptible-infection	LINEAR: cancer are highly susceptible to infection	COUNT: 20
PATH: cancer-patient-susceptible-infection	LINEAR: cancer are highly susceptible to infection	COUNT: 20
PATH: cancer-patient-susceptible-infection	LINEAR: Cancer patients , particularly those under anticancer treatment , are potentially more susceptible for a severe infection	COUNT: 20
PATH: cancer-patient-susceptible-infection	LINEAR: cancer are more susceptible to the infection	COUNT: 20
PATH: cancer-patient-susceptible-infection	LINEAR: Cancer patients and immunocompromised hosts are particularly susceptible to lung infection	COUNT: 20
PATH: cancer-patient-susceptible-infection	LINEAR: cancer are more susceptible to infection	COUNT: 20
PATH: cancer-patient-susceptible-infection	LINEAR: cancer are generally more susceptible to infections	COUNT: 20
PATH: cancer-patient-susceptible-infec

QUERY 4:
	arg1:[e]paracetamol is $not useful for treating arg2:[e]asthma.

MOST FREQUENT PATTERNS:

PATH: corticosteroid-increase-influenza-infection	LINEAR: corticosteroids may increase the risk of mortality and morbidity ( e.g. secondary infections ) in severe influenza and other viral infections	COUNT: 2
PATH: corticosteroid-increase-influenza-infection	LINEAR: corticosteroids may increase the risk of mortality and morbidity ( e.g. secondary infections ) in severe influenza and other viral infections	COUNT: 2
PATH: ribavirin-consider-management-case-severe	LINEAR: ribavirin and IgIV can be considered for the management of severe	COUNT: 2
PATH: ribavirin-effective-virus-sars	LINEAR: ribavirin is only marginally effective against the SARS	COUNT: 2
PATH: antiviral-option-limited-be-prove-effective-occur-disease	LINEAR: antiviral treatment options are limited because no parenteral drug is available and no drug has been proved to be effective once life-threatening disease	COUNT: 2
PATH: 

QUERY 5:
	arg1:[e]stroke is a $complication of :[e]COVID-19 arg2:infection

MOST FREQUENT PATTERNS:

PATH: disease-be	LINEAR: is a novel respiratory disease	COUNT: 7
PATH: disease-be	LINEAR: is a novel respiratory disease	COUNT: 7
PATH: disease-be	LINEAR: is a highly contagious disease	COUNT: 7
PATH: disease-be	LINEAR: is an infectious disease	COUNT: 7
PATH: disease-be	LINEAR: is an acute infectious disease	COUNT: 7
PATH: disease-be	LINEAR: is an acute resolved disease	COUNT: 7
PATH: disease-be	LINEAR: is a common cardiovascular disease	COUNT: 7
PATH: infection-patient	LINEAR: Patients with severe COVID-19 infection	COUNT: 5
PATH: infection-complication-patient	LINEAR: Infection is a common complication in patients	COUNT: 5
PATH: infection-complication-patient	LINEAR: infection is a known complication among critically ill patients	COUNT: 5
PATH: infection-patient	LINEAR: Patients with invasive GAS infection	COUNT: 5
PATH: infection-complication-patient	LINEAR: infection ( varicella ) i

QUERY 6:
	the recommended arg1:[w]quarantine period is arg2:14 :[w]days

MOST FREQUENT PATTERNS:

PATH: quarantine-for-day-14	LINEAR: quarantined for 14	COUNT: 7
PATH: quarantine-for-day-14	LINEAR: quarantined for 14	COUNT: 7
PATH: quarantine-for-day-14	LINEAR: quarantined for 14	COUNT: 7
PATH: quarantine-for-day-14	LINEAR: quarantined for 14	COUNT: 7
PATH: quarantine-for-day-14	LINEAR: quarantined for 14	COUNT: 7
PATH: quarantine-for-day-14	LINEAR: quarantined for 14	COUNT: 7
PATH: quarantine-for-day-14	LINEAR: quarantine for 14	COUNT: 7
PATH: quarantine-period	LINEAR: quarantine period	COUNT: 6
PATH: quarantine-a	LINEAR: a 14-day quarantine	COUNT: 6
PATH: quarantine-period	LINEAR: quarantine period	COUNT: 6
PATH: quarantine-a	LINEAR: A 14-day quarantine	COUNT: 6
PATH: quarantine-a	LINEAR: A 14‐day quarantine	COUNT: 6
PATH: quarantine-period	LINEAR: quarantine period	COUNT: 6
PATH: quarantine-a	LINEAR: a 2-week quarantine	COUNT: 6
PATH: quarantine-a	LINEAR: a 14-day quarantine	COUNT: 

QUERY 7:
	arg1:[e]COVID-19 $infects arg2:cells

MOST FREQUENT PATTERNS:

PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 infects lung alveolar epithelial cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 infects cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 infects ACE2positive cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 infects human epithelial cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 can infect cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 infects endothelial cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 infects host cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 infects host cell	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 infects cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-COV-2 infects human cells	COUNT: 21
PATH: sars-cov-2-infect-cell	LINEAR: SARS-CoV-2 mainly infects type II alveolar epithelial cells	COUNT: 21
PATH: sars-cov-

QUERY 8:
	arg1:[e]COVID-19 activates the arg2:ATP $receptor

MOST FREQUENT PATTERNS:

PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: Enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT: 24
PATH: 2-enzyme-ace2	LINEAR: enzyme 2 ( ACE2	COUNT:

## Pattern: dep labels along dep path

In [66]:
collect(500, "deps")

QUERY 0:
	<>arg1:virus $infection $causes a <>arg2:condition .

MOST FREQUENT PATTERNS:

PATH: nsubj-ROOT-dobj	LINEAR: virus , like MHV-3 infection in mice , can induce exaggerated inflammation	COUNT: 59
PATH: nsubj-ROOT-dobj	LINEAR: infections cause a debilitating disease	COUNT: 59
PATH: nsubj-ROOT-dobj	LINEAR: MERS-CoV induces acute pneumonia	COUNT: 59
PATH: nsubj-ROOT-dobj	LINEAR: infection with HHV-6 can produce a mononucleosis-like illness and , more rarely , severe disease	COUNT: 59
PATH: nsubj-ROOT-dobj	LINEAR: infection , in domestic cats , causes a variable immunodeficiency syndrome	COUNT: 59
PATH: nsubj-ROOT-dobj	LINEAR: virus causes a lower respiratory tract disease	COUNT: 59
PATH: nsubj-ROOT-dobj	LINEAR: MERS-CoV causes severe infection	COUNT: 59
PATH: nsubj-ROOT-dobj	LINEAR: SARS-CoV causes interstitial pneumonia	COUNT: 59
PATH: nsubj-ROOT-dobj	LINEAR: infection causes myocarditis	COUNT: 59
PATH: nsubj-ROOT-dobj	LINEAR: infection causes undifferentiated febrile illness	COU

QUERY 1:
	infected arg1:patients $usually $develop <>arg2:[entity]respiratory illness.

MOST FREQUENT PATTERNS:

PATH: nsubj-ROOT-dobj	LINEAR: patients who have pneumonic plague typically develop fulminant pneumonia	COUNT: 33
PATH: nsubj-ROOT-dobj	LINEAR: Humans infected with RVFV usually develop an acute febrile myalgic syndrome	COUNT: 33
PATH: nsubj-ROOT-dobj	LINEAR: Patients with bubonic plague usually develop symptoms	COUNT: 33
PATH: nsubj-ROOT-dobj	LINEAR: patients develop rapid onset fever	COUNT: 33
PATH: nsubj-ROOT-dobj	LINEAR: patients with infectious mononucleosis usually experience an abrupt onset	COUNT: 33
PATH: nsubj-ROOT-dobj	LINEAR: Cats with FIP develop lesions	COUNT: 33
PATH: nsubj-ROOT-dobj	LINEAR: horses have a mild illness	COUNT: 33
PATH: nsubj-ROOT-dobj	LINEAR: Rabbits infected with RDHV often develop acute necrotizing hepatitis	COUNT: 33
PATH: nsubj-ROOT-dobj	LINEAR: individuals who die within 10 days of the onset of symptoms show diffuse alveolar damage	COUNT: 33


QUERY 2:
	a arg1:subset of patients $progress to arg2:hemorrhagic fever

MOST FREQUENT PATTERNS:

PATH: nsubj-ROOT-nmod-amod	LINEAR: severe cases , the disease may progress	COUNT: 51
PATH: nsubj-ROOT-nmod-amod	LINEAR: severe cases , the illness can progress	COUNT: 51
PATH: nsubj-ROOT-nmod-amod	LINEAR: severe cases , the pneumonia rapidly progresses	COUNT: 51
PATH: nsubj-ROOT-nmod-amod	LINEAR: critical patients , the disease progresses	COUNT: 51
PATH: nsubj-ROOT-nmod-amod	LINEAR: Symptoms can progress to a severe	COUNT: 51
PATH: nsubj-ROOT-nmod-amod	LINEAR: disease may progress into respiratory	COUNT: 51
PATH: nsubj-ROOT-nmod-amod	LINEAR: illness can progress to severe	COUNT: 51
PATH: nsubj-ROOT-nmod-amod	LINEAR: disease quickly progresses to respiratory	COUNT: 51
PATH: nsubj-ROOT-nmod-amod	LINEAR: disease quickly progresses to respiratory	COUNT: 51
PATH: nsubj-ROOT-nmod-amod	LINEAR: severe cases , the disease can lead	COUNT: 51
PATH: nsubj-ROOT-nmod-amod	LINEAR: IPS may rapidly progres

QUERY 3:
	people with arg1:[e]asthma are $susceptible to arg2:[e]asthma

MOST FREQUENT PATTERNS:

PATH: nsubj-ROOT-nmod	LINEAR: patients , including those with HIV/AIDS and patients taking immunosuppressants post-transplant , are at an increased risk for viral and fungal pneumonia	COUNT: 43
PATH: nsubj-ROOT-nmod	LINEAR: Patients with immunodeficiency due to human immunodeficiency virus ( HIV ) infection are highly susceptible to viral sepsis	COUNT: 43
PATH: nsubj-ROOT-nmod	LINEAR: patients are also more prone to RSV infections	COUNT: 43
PATH: nsubj-ROOT-nmod	LINEAR: patients are more vulnerable to infection	COUNT: 43
PATH: nsubj-ROOT-nmod	LINEAR: patients are often vulnerable to opportunistic infections	COUNT: 43
PATH: nsubj-ROOT-nmod	LINEAR: children with congenital or acquired immunodeficiencies are more susceptible to viral pathogens	COUNT: 43
PATH: nsubj-ROOT-nmod	LINEAR: Patients with liver cirrhosis or liver cancer are at high risk of SARS-CoV-2 infection	COUNT: 43
PATH: nsubj-RO

QUERY 4:
	arg1:[e]paracetamol is $not useful for treating arg2:[e]asthma.

MOST FREQUENT PATTERNS:

PATH: nsubj-ROOT-nmod-nmod	LINEAR: steroids are not effective for the treatment of lung injury or shock in adults ' SARS-CoV-2 infection	COUNT: 5
PATH: nsubj-ROOT-nmod-nmod	LINEAR: Ganciclovir is the treatment of choice for invasive CMV disease	COUNT: 5
PATH: nsubj-ROOT-nmod-nmod	LINEAR: corticosteroids may provide benefi t to certain subgroups of patients	COUNT: 5
PATH: nsubj-ROOT-nmod-nmod	LINEAR: therapy is not possible in many patients either because disease-specific therapy may not be available , available disease-specific therapies may have failed , or due to late presentation with established cirrhosis	COUNT: 5
PATH: nsubj-ROOT-nmod-nmod	LINEAR: Corticosteroids are the mainstay of treatment for acute GVHD	COUNT: 5
PATH: nsubjpass-ROOT-nmod	LINEAR: steroids are not recommended in COVID-19 infection	COUNT: 4
PATH: nsubj-ROOT-nmod-nmod-compound	LINEAR: Drugs currently licensed for th

QUERY 5:
	arg1:[e]stroke is a $complication of :[e]COVID-19 arg2:infection

MOST FREQUENT PATTERNS:

PATH: nsubj-ROOT-nmod	LINEAR: pneumothorax is a specific and potentially life-threatening complication in SARS patients	COUNT: 57
PATH: nsubj-ROOT-nmod	LINEAR: injury , which significantly increases mortality , is often evident in patients	COUNT: 57
PATH: nsubj-ROOT-nmod	LINEAR: IH is a significant complication of high-dose IVIg infusion	COUNT: 57
PATH: nsubj-ROOT-nmod	LINEAR: failure can be the primary presenting manifestation of COVID-19 infection	COUNT: 57
PATH: nsubj-ROOT-nmod	LINEAR: damage is a common feature in severe COVID-19 patients	COUNT: 57
PATH: nsubj-ROOT-nmod	LINEAR: embolism is an uncharacteristic presentation in patients	COUNT: 57
PATH: nsubj-ROOT-nmod	LINEAR: pneumonia is a common complication of VZV septicemia in children	COUNT: 57
PATH: nsubj-ROOT-nmod	LINEAR: pneumonia is a common complication of VZV septicemia in children	COUNT: 57
PATH: nsubj-ROOT-nmod	LINEAR: pne

QUERY 6:
	the recommended arg1:[w]quarantine period is arg2:14 :[w]days

MOST FREQUENT PATTERNS:

PATH: compound-nsubj-ROOT-attr-nummod	LINEAR: quarantine period must be at least 22	COUNT: 11
PATH: compound-nsubj-ROOT-attr-nummod	LINEAR: quarantine period is typically 30	COUNT: 11
PATH: compound-nsubj-ROOT-attr-nummod	LINEAR: exposure time for low-level disinfection of noncritical items is at least 1	COUNT: 11
PATH: compound-nsubj-ROOT-attr-nummod	LINEAR: harvest interval must be at least 30	COUNT: 11
PATH: compound-nsubj-ROOT-attr-nummod	LINEAR: Quarantine period for all felid species is typically 30	COUNT: 11
PATH: compound-nsubj-ROOT-attr-nummod	LINEAR: reporting delay on COVID-19 tests is about 6	COUNT: 11
PATH: compound-nsubj-ROOT-attr-nummod	LINEAR: contact time of items/tools with the disinfectant in the tank is 10	COUNT: 11
PATH: compound-nsubj-ROOT-attr-nummod	LINEAR: epidemic threshold is 10	COUNT: 11
PATH: compound-nsubj-ROOT-attr-nummod	LINEAR: recovery timescale is about 2

QUERY 7:
	arg1:[e]COVID-19 $infects arg2:cells

MOST FREQUENT PATTERNS:

PATH: nsubj-ROOT-dobj	LINEAR: SARS-CoV-2 infects lung alveolar epithelial cells	COUNT: 76
PATH: nsubj-ROOT-dobj	LINEAR: SARS-CoV-2 infects cells	COUNT: 76
PATH: nsubj-ROOT-dobj	LINEAR: SARA-CoV-2 primarily invades alveolar epithelial cells	COUNT: 76
PATH: nsubj-ROOT-dobj	LINEAR: SARS-CoV-2 infects ACE2positive cells	COUNT: 76
PATH: nsubj-ROOT-dobj	LINEAR: SARS-CoV-2 invades target cells	COUNT: 76
PATH: nsubj-ROOT-dobj	LINEAR: SARS-CoV-2 infects human epithelial cells	COUNT: 76
PATH: nsubj-ROOT-dobj	LINEAR: SARS-CoV-2 can infect cells	COUNT: 76
PATH: nsubj-ROOT-dobj	LINEAR: SARS-Cov-2 first predominantly infects lower airways	COUNT: 76
PATH: nsubj-ROOT-dobj	LINEAR: SARS-CoV-2 enters human cells	COUNT: 76
PATH: nsubj-ROOT-dobj	LINEAR: SARS-CoV-2 enters respiratory epithelial cells	COUNT: 76
PATH: nsubj-ROOT-dobj	LINEAR: SARS-CoV-2 enters human cells	COUNT: 76
PATH: nsubj-ROOT-dobj	LINEAR: SARS-CoV-2 infects endothel

QUERY 8:
	arg1:[e]COVID-19 activates the arg2:ATP $receptor

MOST FREQUENT PATTERNS:

PATH: nsubj-ROOT-dobj-appos	LINEAR: SARS-CoV-2 employs angiotensin-converting enzyme-2 ( ACE2	COUNT: 26
PATH: nsubj-ROOT-dobj-appos	LINEAR: SARS-CoV-2 exploits angiotensin-converting enzyme 2 ( ACE2	COUNT: 26
PATH: nsubj-ROOT-dobj-appos	LINEAR: SARS-CoV-2 employs angiotensin-converting enzyme 2 ( ACE2	COUNT: 26
PATH: nsubj-ROOT-dobj-appos	LINEAR: SARS-CoV-2 uses angiotensin-converting enzyme 2 ( ACE2	COUNT: 26
PATH: nsubj-ROOT-dobj-appos	LINEAR: SARS-CoV-2 and SARS-CoV use angiotensin-converting enzyme 2 ( ACE2	COUNT: 26
PATH: nsubj-ROOT-dobj-appos	LINEAR: SARS-CoV-2 , like SARS-CoV , uses the same receptor angiotensin converting enzyme 2 ( ACE2	COUNT: 26
PATH: nsubj-ROOT-dobj-appos	LINEAR: SARS-CoV-2 , like SARS-CoV ( Li et al. , 2003 ) , uses the angiotensin converting enzyme 2 ( ACE2	COUNT: 26
PATH: nsubj-ROOT-dobj-appos	LINEAR: SARS-CoV-2 also uses human angiotensin-converting enzyme 2 ( ACE2	COUN

## Pattern: dep labels along dep path (pyBART)

In [67]:
collect(500, "deps", parse_type="pybart")

QUERY 0:
	<>arg1:virus $infection $causes a <>arg2:condition .

MOST FREQUENT PATTERNS:

PATH: nsubj-root-dobj	LINEAR: virus , like MHV-3 infection in mice , can induce exaggerated inflammation	COUNT: 59
PATH: nsubj-root-dobj	LINEAR: infections cause a debilitating disease	COUNT: 59
PATH: nsubj-root-dobj	LINEAR: MERS-CoV induces acute pneumonia	COUNT: 59
PATH: nsubj-root-dobj	LINEAR: infection with HHV-6 can produce a mononucleosis-like illness and , more rarely , severe disease	COUNT: 59
PATH: nsubj-root-dobj	LINEAR: infection , in domestic cats , causes a variable immunodeficiency syndrome	COUNT: 59
PATH: nsubj-root-dobj	LINEAR: virus causes a lower respiratory tract disease	COUNT: 59
PATH: nsubj-root-dobj	LINEAR: MERS-CoV causes severe infection	COUNT: 59
PATH: nsubj-root-dobj	LINEAR: SARS-CoV causes interstitial pneumonia	COUNT: 59
PATH: nsubj-root-dobj	LINEAR: infection causes myocarditis	COUNT: 59
PATH: nsubj-root-dobj	LINEAR: infection causes undifferentiated febrile illness	COU

QUERY 1:
	infected arg1:patients $usually $develop <>arg2:[entity]respiratory illness.

MOST FREQUENT PATTERNS:

PATH: nsubj-root-dobj	LINEAR: patients who have pneumonic plague typically develop fulminant pneumonia	COUNT: 33
PATH: nsubj-root-dobj	LINEAR: Humans infected with RVFV usually develop an acute febrile myalgic syndrome	COUNT: 33
PATH: nsubj-root-dobj	LINEAR: Patients with bubonic plague usually develop symptoms	COUNT: 33
PATH: nsubj-root-dobj	LINEAR: patients develop rapid onset fever	COUNT: 33
PATH: nsubj-root-dobj	LINEAR: patients with infectious mononucleosis usually experience an abrupt onset	COUNT: 33
PATH: nsubj-root-dobj	LINEAR: Cats with FIP develop lesions	COUNT: 33
PATH: nsubj-root-dobj	LINEAR: horses have a mild illness	COUNT: 33
PATH: nsubj-root-dobj	LINEAR: Rabbits infected with RDHV often develop acute necrotizing hepatitis	COUNT: 33
PATH: nsubj-root-dobj	LINEAR: individuals who die within 10 days of the onset of symptoms show diffuse alveolar damage	COUNT: 33


QUERY 2:
	a arg1:subset of patients $progress to arg2:hemorrhagic fever

MOST FREQUENT PATTERNS:

PATH: nsubj-amod	LINEAR: Severe cases	COUNT: 29
PATH: nsubj-amod	LINEAR: Severe cases	COUNT: 29
PATH: nsubj-amod	LINEAR: Mild disease	COUNT: 29
PATH: nsubj-amod	LINEAR: Severe pediatric cases	COUNT: 29
PATH: nsubj-amod	LINEAR: Severe patients	COUNT: 29
PATH: nsubj-amod	LINEAR: Severe patients	COUNT: 29
PATH: nsubj-amod	LINEAR: Severe cases	COUNT: 29
PATH: nsubj-amod	LINEAR: Severe illnesses	COUNT: 29
PATH: nsubj-amod	LINEAR: severe cases	COUNT: 29
PATH: nsubj-amod	LINEAR: Acute infection	COUNT: 29
PATH: nsubj-amod	LINEAR: Severe pneumonia	COUNT: 29
PATH: nsubj-amod	LINEAR: Severe disease	COUNT: 29
PATH: nsubj-amod	LINEAR: Severe patients	COUNT: 29
PATH: nsubj-amod	LINEAR: Systemic manifestations	COUNT: 29
PATH: nsubj-amod	LINEAR: Acute fulminant cases	COUNT: 29
PATH: nsubj-amod	LINEAR: Severe patients	COUNT: 29
PATH: nsubj-amod	LINEAR: Chronic infection	COUNT: 29
PATH: nsubj-amod	LINEAR: V

QUERY 3:
	people with arg1:[e]asthma are $susceptible to arg2:[e]asthma

MOST FREQUENT PATTERNS:

PATH: nmod:with-nsubj-amod-nmod:to	LINEAR: cancer are highly susceptible to infection	COUNT: 25
PATH: nmod:with-nsubj-amod-nmod:to	LINEAR: cancer are highly susceptible to infection	COUNT: 25
PATH: nmod:with-nsubj-amod-nmod:to	LINEAR: SCD are strongly susceptible to encapsulated bacterial organisms	COUNT: 25
PATH: nmod:with-nsubj-amod-nmod:to	LINEAR: diabetes , immunosuppressive disorders , and liver disease including hemochromatosis and alcoholic liver disease are especially susceptible to infection	COUNT: 25
PATH: nmod:with-nsubj-amod-nmod:to	LINEAR: cancer are more susceptible to the infection	COUNT: 25
PATH: nsubj-amod-nmod:to	LINEAR: Patients with immunodeficiency due to human immunodeficiency virus ( HIV ) infection are highly susceptible to viral sepsis	COUNT: 25
PATH: nsubj-amod-nmod:to	LINEAR: patients are also more prone to RSV infections	COUNT: 25
PATH: nsubj-amod-nmod:to	LINEAR

QUERY 4:
	arg1:[e]paracetamol is $not useful for treating arg2:[e]asthma.

MOST FREQUENT PATTERNS:

PATH: nsubj-amod-ev	LINEAR: steroids and immunosuppressive drugs are not universally effective	COUNT: 4
PATH: nsubj-amod-ev	LINEAR: patients are refractory	COUNT: 4
PATH: nsubj-amod-ev	LINEAR: cidofovir is only moderately effective	COUNT: 4
PATH: nsubj-amod-ev	LINEAR: therapies involving sunitinib and regorafenib are also not effective	COUNT: 4
PATH: nmod:of-nsubj-root-nmod:in	LINEAR: HBIG to oral antiviral agents may not provide additional benefit in this setting	COUNT: 3
PATH: nmod:with-nsubj	LINEAR: Treatment with new agents	COUNT: 3
PATH: nsubjpass-root-auxpass	LINEAR: ganciclovir is associated	COUNT: 3
PATH: nmod:with-nsubj	LINEAR: treatment with linezolid	COUNT: 3
PATH: nsubjpass-root-auxpass	LINEAR: treatment is compounded	COUNT: 3
PATH: nmod:with-nsubj	LINEAR: treatment with infliximab	COUNT: 3
PATH: nmod:of-nsubjpass-root-auxpass	LINEAR: therapy is complicated	COUNT: 3
PATH: nmo

QUERY 5:
	arg1:[e]stroke is a $complication of :[e]COVID-19 arg2:infection

MOST FREQUENT PATTERNS:

PATH: nsubj-xcomp-nmod:in	LINEAR: pneumothorax is a specific and potentially life-threatening complication in SARS patients	COUNT: 29
PATH: nsubj-xcomp-nmod:in	LINEAR: damage is a common feature in severe COVID-19 patients	COUNT: 29
PATH: nsubj-xcomp-nmod:in	LINEAR: embolism is an uncharacteristic presentation in patients	COUNT: 29
PATH: nsubj-xcomp-nmod:in	LINEAR: pneumonia is a common complication of VZV septicemia in children	COUNT: 29
PATH: nsubj-xcomp-nmod:in	LINEAR: pneumonia is a common complication of VZV septicemia in children	COUNT: 29
PATH: nsubj-xcomp-nmod:in	LINEAR: Infection is a common complication in patients	COUNT: 29
PATH: nsubj-xcomp-nmod:in	LINEAR: Aspergillus is a common superinfection in critically ill influenza patients	COUNT: 29
PATH: nsubj-xcomp-nmod:in	LINEAR: hemorrhage with disseminated intravascular coagulopathy ( DIC ) is another fatal and unusual complicat

QUERY 6:
	the recommended arg1:[w]quarantine period is arg2:14 :[w]days

MOST FREQUENT PATTERNS:

PATH: compound-nsubj-root-attr-nummod	LINEAR: quarantine period must be at least 22	COUNT: 11
PATH: compound-nsubj-root-attr-nummod	LINEAR: quarantine period is typically 30	COUNT: 11
PATH: compound-nsubj-root-attr-nummod	LINEAR: exposure time for low-level disinfection of noncritical items is at least 1	COUNT: 11
PATH: compound-nsubj-root-attr-nummod	LINEAR: harvest interval must be at least 30	COUNT: 11
PATH: compound-nsubj-root-attr-nummod	LINEAR: Quarantine period for all felid species is typically 30	COUNT: 11
PATH: compound-nsubj-root-attr-nummod	LINEAR: reporting delay on COVID-19 tests is about 6	COUNT: 11
PATH: compound-nsubj-root-attr-nummod	LINEAR: contact time of items/tools with the disinfectant in the tank is 10	COUNT: 11
PATH: compound-nsubj-root-attr-nummod	LINEAR: epidemic threshold is 10	COUNT: 11
PATH: compound-nsubj-root-attr-nummod	LINEAR: recovery timescale is about 2

QUERY 7:
	arg1:[e]COVID-19 $infects arg2:cells

MOST FREQUENT PATTERNS:

PATH: nsubj-root-dobj	LINEAR: SARS-CoV-2 infects lung alveolar epithelial cells	COUNT: 76
PATH: nsubj-root-dobj	LINEAR: SARS-CoV-2 infects cells	COUNT: 76
PATH: nsubj-root-dobj	LINEAR: SARA-CoV-2 primarily invades alveolar epithelial cells	COUNT: 76
PATH: nsubj-root-dobj	LINEAR: SARS-CoV-2 infects ACE2positive cells	COUNT: 76
PATH: nsubj-root-dobj	LINEAR: SARS-CoV-2 invades target cells	COUNT: 76
PATH: nsubj-root-dobj	LINEAR: SARS-CoV-2 infects human epithelial cells	COUNT: 76
PATH: nsubj-root-dobj	LINEAR: SARS-CoV-2 can infect cells	COUNT: 76
PATH: nsubj-root-dobj	LINEAR: SARS-Cov-2 first predominantly infects lower airways	COUNT: 76
PATH: nsubj-root-dobj	LINEAR: SARS-CoV-2 enters human cells	COUNT: 76
PATH: nsubj-root-dobj	LINEAR: SARS-CoV-2 enters respiratory epithelial cells	COUNT: 76
PATH: nsubj-root-dobj	LINEAR: SARS-CoV-2 enters human cells	COUNT: 76
PATH: nsubj-root-dobj	LINEAR: SARS-CoV-2 infects endothel

QUERY 8:
	arg1:[e]COVID-19 activates the arg2:ATP $receptor

MOST FREQUENT PATTERNS:

PATH: nsubj-root-dobj-appos	LINEAR: SARS-CoV-2 employs angiotensin-converting enzyme-2 ( ACE2	COUNT: 26
PATH: nsubj-root-dobj-appos	LINEAR: SARS-CoV-2 exploits angiotensin-converting enzyme 2 ( ACE2	COUNT: 26
PATH: nsubj-root-dobj-appos	LINEAR: SARS-CoV-2 employs angiotensin-converting enzyme 2 ( ACE2	COUNT: 26
PATH: nsubj-root-dobj-appos	LINEAR: SARS-CoV-2 uses angiotensin-converting enzyme 2 ( ACE2	COUNT: 26
PATH: nsubj-root-dobj-appos	LINEAR: SARS-CoV-2 and SARS-CoV use angiotensin-converting enzyme 2 ( ACE2	COUNT: 26
PATH: nsubj-root-dobj-appos	LINEAR: SARS-CoV-2 , like SARS-CoV , uses the same receptor angiotensin converting enzyme 2 ( ACE2	COUNT: 26
PATH: nsubj-root-dobj-appos	LINEAR: SARS-CoV-2 , like SARS-CoV ( Li et al. , 2003 ) , uses the angiotensin converting enzyme 2 ( ACE2	COUNT: 26
PATH: nsubj-root-dobj-appos	LINEAR: SARS-CoV-2 also uses human angiotensin-converting enzyme 2 ( ACE2	COUN

# Pattern: dep labels + root lemma

In [20]:
collect(500, "lemmas-deps-root")

QUERY 0:
	<>arg1:virus $infection $causes a <>arg2:condition .

MOST FREQUENT PATTERNS:

PATH: nsubj-cause.ROOT-dobj	LINEAR: infections cause a debilitating disease	COUNT: 41
PATH: nsubj-cause.ROOT-dobj	LINEAR: infection , in domestic cats , causes a variable immunodeficiency syndrome	COUNT: 41
PATH: nsubj-cause.ROOT-dobj	LINEAR: virus causes a lower respiratory tract disease	COUNT: 41
PATH: nsubj-cause.ROOT-dobj	LINEAR: MERS-CoV causes severe infection	COUNT: 41
PATH: nsubj-cause.ROOT-dobj	LINEAR: SARS-CoV causes interstitial pneumonia	COUNT: 41
PATH: nsubj-cause.ROOT-dobj	LINEAR: infection causes myocarditis	COUNT: 41
PATH: nsubj-cause.ROOT-dobj	LINEAR: infection causes undifferentiated febrile illness	COUNT: 41
PATH: nsubj-cause.ROOT-dobj	LINEAR: infection causes a sudden onset	COUNT: 41
PATH: nsubj-cause.ROOT-dobj	LINEAR: MARV causes an acute febrile illness	COUNT: 41
PATH: nsubj-cause.ROOT-dobj	LINEAR: infection causes severe atrophic enteritis	COUNT: 41
PATH: nsubj-cause.ROOT-dob

QUERY 1:
	infected arg1:patients $usually $develop <>arg2:[entity]respiratory illness.

MOST FREQUENT PATTERNS:

PATH: nsubj-develop.ROOT-dobj	LINEAR: patients who have pneumonic plague typically develop fulminant pneumonia	COUNT: 18
PATH: nsubj-develop.ROOT-dobj	LINEAR: Humans infected with RVFV usually develop an acute febrile myalgic syndrome	COUNT: 18
PATH: nsubj-develop.ROOT-dobj	LINEAR: Patients with bubonic plague usually develop symptoms	COUNT: 18
PATH: nsubj-develop.ROOT-dobj	LINEAR: patients develop rapid onset fever	COUNT: 18
PATH: nsubj-develop.ROOT-dobj	LINEAR: Cats with FIP develop lesions	COUNT: 18
PATH: nsubj-develop.ROOT-dobj	LINEAR: Rabbits infected with RDHV often develop acute necrotizing hepatitis	COUNT: 18
PATH: nsubj-develop.ROOT-dobj	LINEAR: horses develop acute , febrile respiratory disease	COUNT: 18
PATH: nsubj-develop.ROOT-dobj	LINEAR: sheep develop chronic diarrhea	COUNT: 18
PATH: nsubj-develop.ROOT-dobj	LINEAR: hosts usually develop self-limiting diarrhea	C

QUERY 2:
	a arg1:subset of patients $progress to arg2:hemorrhagic fever

MOST FREQUENT PATTERNS:

PATH: nsubj-progress.ROOT-nmod-amod	LINEAR: severe cases , the disease may progress	COUNT: 31
PATH: nsubj-progress.ROOT-nmod-amod	LINEAR: severe cases , the illness can progress	COUNT: 31
PATH: nsubj-progress.ROOT-nmod-amod	LINEAR: severe cases , the pneumonia rapidly progresses	COUNT: 31
PATH: nsubj-progress.ROOT-nmod-amod	LINEAR: critical patients , the disease progresses	COUNT: 31
PATH: nsubj-progress.ROOT-nmod-amod	LINEAR: Symptoms can progress to a severe	COUNT: 31
PATH: nsubj-progress.ROOT-nmod-amod	LINEAR: disease may progress into respiratory	COUNT: 31
PATH: nsubj-progress.ROOT-nmod-amod	LINEAR: illness can progress to severe	COUNT: 31
PATH: nsubj-progress.ROOT-nmod-amod	LINEAR: disease quickly progresses to respiratory	COUNT: 31
PATH: nsubj-progress.ROOT-nmod-amod	LINEAR: disease quickly progresses to respiratory	COUNT: 31
PATH: nsubj-progress.ROOT-nmod-amod	LINEAR: IPS may rapidl

QUERY 3:
	people with arg1:[e]asthma are $susceptible to arg2:[e]asthma

MOST FREQUENT PATTERNS:

PATH: nmod-nsubj-susceptible.ROOT-nmod	LINEAR: cancer are highly susceptible to infection	COUNT: 20
PATH: nmod-nsubj-susceptible.ROOT-nmod	LINEAR: cancer are highly susceptible to infection	COUNT: 20
PATH: nmod-nsubj-susceptible.ROOT-nmod	LINEAR: SCD are strongly susceptible to encapsulated bacterial organisms	COUNT: 20
PATH: nmod-nsubj-susceptible.ROOT-nmod	LINEAR: diabetes , immunosuppressive disorders , and liver disease including hemochromatosis and alcoholic liver disease are especially susceptible to infection	COUNT: 20
PATH: nmod-nsubj-susceptible.ROOT-nmod	LINEAR: cancer are more susceptible to the infection	COUNT: 20
PATH: nmod-nsubj-susceptible.ROOT-nmod	LINEAR: cancer are more susceptible to infection	COUNT: 20
PATH: nmod-nsubj-susceptible.ROOT-nmod	LINEAR: fibrosis are particularly susceptible to chronic infection	COUNT: 20
PATH: nmod-nsubj-susceptible.ROOT-nmod	LINEAR: COPD ar

QUERY 4:
	arg1:[e]paracetamol is $not useful for treating arg2:[e]asthma.

MOST FREQUENT PATTERNS:

PATH: nsubj-effective.ROOT-cop	LINEAR: steroids and immunosuppressive drugs are not universally effective	COUNT: 3
PATH: nmod-treatment.nsubj	LINEAR: Treatment with new agents	COUNT: 3
PATH: nmod-treatment.nsubj	LINEAR: treatment with linezolid	COUNT: 3
PATH: nsubj-effective.ROOT-cop	LINEAR: cidofovir is only moderately effective	COUNT: 3
PATH: nmod-treatment.nsubj	LINEAR: treatment with infliximab	COUNT: 3
PATH: nmod-dobj-require.ROOT-nsubj	LINEAR: Patients with CF and ABPA often require prolonged therapy with oral corticosteroids	COUNT: 3
PATH: nsubj-effective.ROOT-cop	LINEAR: therapies involving sunitinib and regorafenib are also not effective	COUNT: 3
PATH: nmod-dobj-require.ROOT-nsubj	LINEAR: patients require immediate immunosuppression with corticosteroids	COUNT: 3
PATH: nmod-dobj-require.ROOT-nsubj	LINEAR: patients often require prolonged therapy with ganciclovir	COUNT: 3
PATH: ns

QUERY 5:
	arg1:[e]stroke is a $complication of :[e]COVID-19 arg2:infection

MOST FREQUENT PATTERNS:

PATH: nsubj-complication.ROOT-nmod	LINEAR: pneumothorax is a specific and potentially life-threatening complication in SARS patients	COUNT: 26
PATH: nsubj-complication.ROOT-nmod	LINEAR: IH is a significant complication of high-dose IVIg infusion	COUNT: 26
PATH: nsubj-complication.ROOT-nmod	LINEAR: pneumonia is a common complication of VZV septicemia in children	COUNT: 26
PATH: nsubj-complication.ROOT-nmod	LINEAR: pneumonia is a common complication of VZV septicemia in children	COUNT: 26
PATH: nsubj-complication.ROOT-nmod	LINEAR: pneumonia ( PVP ) is a rare complication of seasonal influenza	COUNT: 26
PATH: nsubj-complication.ROOT-nmod	LINEAR: pneumonia is a common complication of respiratory virus infection	COUNT: 26
PATH: nsubj-complication.ROOT-nmod	LINEAR: ILD is a rare but potentially fatal complication of RTX therapy	COUNT: 26
PATH: nsubj-complication.ROOT-nmod	LINEAR: pneumonia is

QUERY 6:
	the recommended arg1:[w]quarantine period is arg2:14 :[w]days

MOST FREQUENT PATTERNS:

PATH: compound-nsubj-be.ROOT-attr-nummod	LINEAR: quarantine period must be at least 22	COUNT: 11
PATH: compound-nsubj-be.ROOT-attr-nummod	LINEAR: quarantine period is typically 30	COUNT: 11
PATH: compound-nsubj-be.ROOT-attr-nummod	LINEAR: exposure time for low-level disinfection of noncritical items is at least 1	COUNT: 11
PATH: compound-nsubj-be.ROOT-attr-nummod	LINEAR: harvest interval must be at least 30	COUNT: 11
PATH: compound-nsubj-be.ROOT-attr-nummod	LINEAR: Quarantine period for all felid species is typically 30	COUNT: 11
PATH: compound-nsubj-be.ROOT-attr-nummod	LINEAR: reporting delay on COVID-19 tests is about 6	COUNT: 11
PATH: compound-nsubj-be.ROOT-attr-nummod	LINEAR: contact time of items/tools with the disinfectant in the tank is 10	COUNT: 11
PATH: compound-nsubj-be.ROOT-attr-nummod	LINEAR: epidemic threshold is 10	COUNT: 11
PATH: compound-nsubj-be.ROOT-attr-nummod	LINEAR: re

QUERY 7:
	arg1:[e]COVID-19 $infects arg2:cells

MOST FREQUENT PATTERNS:

PATH: nsubj-infect.ROOT-dobj	LINEAR: SARS-CoV-2 infects lung alveolar epithelial cells	COUNT: 39
PATH: nsubj-infect.ROOT-dobj	LINEAR: SARS-CoV-2 infects cells	COUNT: 39
PATH: nsubj-infect.ROOT-dobj	LINEAR: SARS-CoV-2 infects ACE2positive cells	COUNT: 39
PATH: nsubj-infect.ROOT-dobj	LINEAR: SARS-CoV-2 infects human epithelial cells	COUNT: 39
PATH: nsubj-infect.ROOT-dobj	LINEAR: SARS-CoV-2 can infect cells	COUNT: 39
PATH: nsubj-infect.ROOT-dobj	LINEAR: SARS-Cov-2 first predominantly infects lower airways	COUNT: 39
PATH: nsubj-infect.ROOT-dobj	LINEAR: SARS-CoV-2 infects endothelial cells	COUNT: 39
PATH: nsubj-infect.ROOT-dobj	LINEAR: SARS-Cov-2 first predominantly infects lower airways	COUNT: 39
PATH: nsubj-infect.ROOT-dobj	LINEAR: SARS-CoV2 infects lung alveolar epithelial cells	COUNT: 39
PATH: nsubj-infect.ROOT-dobj	LINEAR: SARS-CoV-2 infects host cells	COUNT: 39
PATH: nsubj-infect.ROOT-dobj	LINEAR: SARS-CoV-2 infe

QUERY 8:
	arg1:[e]COVID-19 activates the arg2:ATP $receptor

MOST FREQUENT PATTERNS:

PATH: nsubj-use.ROOT-dobj-appos	LINEAR: SARS-CoV-2 uses angiotensin-converting enzyme 2 ( ACE2	COUNT: 17
PATH: nsubj-use.ROOT-dobj-appos	LINEAR: SARS-CoV-2 and SARS-CoV use angiotensin-converting enzyme 2 ( ACE2	COUNT: 17
PATH: nsubj-use.ROOT-dobj-appos	LINEAR: SARS-CoV-2 , like SARS-CoV , uses the same receptor angiotensin converting enzyme 2 ( ACE2	COUNT: 17
PATH: nsubj-use.ROOT-dobj-appos	LINEAR: SARS-CoV-2 , like SARS-CoV ( Li et al. , 2003 ) , uses the angiotensin converting enzyme 2 ( ACE2	COUNT: 17
PATH: nsubj-use.ROOT-dobj-appos	LINEAR: SARS-CoV-2 also uses human angiotensin-converting enzyme 2 ( ACE2	COUNT: 17
PATH: nsubj-use.ROOT-dobj-appos	LINEAR: SARS-CoV-2 uses angiotensin-converting enzyme 2 ( ACE2	COUNT: 17
PATH: nsubj-use.ROOT-dobj-appos	LINEAR: SARS-CoV-2 uses angiotensin-converting enzyme 2 ( ACE2	COUNT: 17
PATH: nsubj-use.ROOT-dobj-appos	LINEAR: SARS-CoV-2 can further use transmembr